In [1]:
# df for question answer file csv contains 1 Lakh data points
import pandas as pd
df = pd.read_csv("/kaggle/input/qna-1l/cleaned_qna.csv")

In [2]:
!pip install -q peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.5 MB/s eta 0:00:00:00:0100:01


In [3]:
# to read images.csv from vr-miniproject-2 or one can upload it manually and then copy the relative path
file_path = pd.read_csv("/kaggle/input/vr-miniproject-2/abo-images-small/images/metadata/images.csv")

In [4]:
# merging qna with images
final_df = pd.merge(df, file_path, on='image_id', how='left')

In [5]:
# to select specific col
final_df = final_df[["image_id","path","question","answer"]]

In [6]:
# adjusting the path of images according to kaggle
import os
image_folder = '/kaggle/input/vr-miniproject-2/abo-images-small/images/small/'
final_df['path'] = final_df['path'].apply(lambda p: os.path.join(image_folder, p))

In [7]:
# install bert score package
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00


In [8]:
# install bart score
!git clone https://github.com/neulab/BARTScore.git
import sys
sys.path.append('/kaggle/working/BARTScore')

Cloning into 'BARTScore'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 220 (delta 18), reused 14 (delta 14), pack-reused 194 (from 1)
Receiving objects: 100% (220/220), 101.98 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Updating files: 100% (192/192), done.


In [9]:
# requiered classes to do fine tuning
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [10]:
# change question and answert to lower case
final_df["question"] = final_df["question"].str.lower()
final_df["answer"] = final_df["answer"].str.lower()

In [11]:
# just to run some sample to test if the model is working or not, if one is sure one can work just remove .iloc[0:1000]
df = final_df.iloc[0:80000]

In [12]:
# change question and answert to lower case
df["question"] = df["question"].str.lower()
df["answer"] = df["answer"].str.lower()

/tmp/ipykernel_35/821518018.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["question"] = df["question"].str.lower()
/tmp/ipykernel_35/821518018.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["answer"] = df["answer"].str.lower()


In [13]:
from sklearn.model_selection import train_test_split

# Data split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [14]:
import os
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForQuestionAnswering, get_scheduler
from peft import get_peft_model, LoraConfig, TaskType

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load processor and base model
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
base_model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

# Patch forward to avoid unwanted args
_original_forward = base_model.forward
def patched_forward(*args, **kwargs):
    kwargs.pop("inputs_embeds", None)
    kwargs.pop("decoder_inputs_embeds", None)
    return _original_forward(*args, **kwargs)

base_model.forward = patched_forward

# Apply LoRA
lora_cfg = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

# Custom dataset class
class VQADataset(Dataset):
    def __init__(self, dataframe, image_folder, processor):
        self.data = dataframe
        self.image_folder = image_folder
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        image_path = os.path.join(self.image_folder, item['path'])
        image = Image.open(image_path).convert("RGB")

        question = str(item['question'])
        answer = str(item['answer'])

        inputs = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding="max_length",
            max_length=128,
            truncation=True
        )

        inputs = {key: val.squeeze(0) for key, val in inputs.items()}
        target_ids = self.processor.tokenizer(
            answer,
            return_tensors="pt",
            padding="max_length",
            max_length=32,
            truncation=True
        ).input_ids.squeeze(0)

        inputs["labels"] = target_ids
        return inputs

# Collate function for dynamic padding
def custom_collate(batch):
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_masks = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.nn.utils.rnn.pad_sequence(
        [item["labels"] for item in batch], batch_first=True, padding_value=-100
    )

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

# Define paths and data
image_folder = "/kaggle/input/vr-miniproject-2/abo-images-small"

# Data loader setup
dataset = VQADataset(train_df, image_folder, processor)
loader = DataLoader(dataset, batch_size=16, shuffle=True, collate_fn=custom_collate, num_workers=2)

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = get_scheduler(
    "cosine", optimizer=optimizer, num_warmup_steps=100, num_training_steps=len(loader) * 3
)

# Mixed precision setup
scaler = torch.cuda.amp.GradScaler()

# Training loop
model.train()
epochs = 5

for epoch in range(epochs):
    epoch_loss = 0
    for step, batch in enumerate(loader):
        pixel_values = batch["pixel_values"].to(device, dtype=torch.float16)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = output.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        epoch_loss += loss.item()
        if step % 10 == 0:
            print(f"Epoch {epoch+1} | Step {step} | Loss: {loss.item():.4f}")

    avg_loss = epoch_loss / len(loader)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")

# Save adapter
output_dir = "/kaggle/working/lora_adapter_blip"
model.save_pretrained(output_dir)
print(f"Model saved at: {output_dir}")

2025-05-14 04:26:55.484782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747196815.660880      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747196815.721067      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Using device: cuda


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

trainable params: 4,718,592 || all params: 389,391,164 || trainable%: 1.2118


/tmp/ipykernel_35/3665242877.py:110: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipykernel_35/3665242877.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Epoch 1 | Step 0 | Loss: 10.4318
Epoch 1 | Step 10 | Loss: 10.2646
Epoch 1 | Step 20 | Loss: 10.3067
Epoch 1 | Step 30 | Loss: 10.1168
Epoch 1 | Step 40 | Loss: 10.1603
Epoch 1 | Step 50 | Loss: 9.9676
Epoch 1 | Step 60 | Loss: 9.7833
Epoch 1 | Step 70 | Loss: 9.6069
Epoch 1 | Step 80 | Loss: 9.4943
Epoch 1 | Step 90 | Loss: 9.3644
Epoch 1 | Step 100 | Loss: 9.2021
Epoch 1 | Step 110 | Loss: 9.0576
Epoch 1 | Step 120 | Loss: 8.9485
Epoch 1 | Step 130 | Loss: 8.8625
Epoch 1 | Step 140 | Loss: 8.7292
Epoch 1 | Step 150 | Loss: 8.7188
Epoch 1 | Step 160 | Loss: 8.6750
Epoch 1 | Step 170 | Loss: 8.6446
Epoch 1 | Step 180 | Loss: 8.5607
Epoch 1 | Step 190 | Loss: 8.5451
Epoch 1 | Step 200 | Loss: 8.5203
Epoch 1 | Step 210 | Loss: 8.4857
Epoch 1 | Step 220 | Loss: 8.4905
Epoch 1 | Step 230 | Loss: 8.4483
Epoch 1 | Step 240 | Loss: 8.3990
Epoch 1 | Step 250 | Loss: 8.4299
Epoch 1 | Step 260 | Loss: 8.3961
Epoch 1 | Step 270 | Loss: 8.3782
Epoch 1 | Step 280 | Loss: 8.3567
Epoch 1 | Step 290 |

In [15]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
import os

# Custom test dataset for image-question pairs
class VQAEvalDataset(Dataset):
    def __init__(self, dataframe, image_dir, processor):
        self.data = dataframe
        self.image_dir = image_dir
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        img_path = os.path.join(self.image_dir, item['path'])
        image = Image.open(img_path).convert("RGB")
        question = str(item['question'])

        encoded = self.processor(
            images=image,
            text=question,
            return_tensors="pt",
            padding="max_length",
            max_length=128,
            truncation=True
        )
        return {key: val.squeeze(0) for key, val in encoded.items()}

# Setup for validation
image_folder = "/kaggle/input/vr-miniproject-2/abo-images-small"
eval_dataset = VQAEvalDataset(val_df, image_folder, processor)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False, num_workers=2)

# Inference mode
model.eval()
generated_answers = []

with torch.no_grad():
    for batch in eval_loader:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        generated_ids = model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10,  # increase answer length
            num_beams=5,         # beam search
            early_stopping=True
        )

        decoded = processor.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        generated_answers.extend(decoded)

# Add predictions to DataFrame
val_df["predicted_answer"] = generated_answers
val_df.head()

,image_id,path,question,answer,predicted_answer
47044,41P5hXfar1L,/kaggle/input/vr-miniproject-2/abo-images-smal...,what is shown on panel?,dimension,handles
44295,41K0IOxW12L,/kaggle/input/vr-miniproject-2/abo-images-smal...,what shape is plug?,rectangular,round
74783,71zl5T96wLL,/kaggle/input/vr-miniproject-2/abo-images-smal...,what is the food item?,bread,almonds
70975,71k-rL1WLvL,/kaggle/input/vr-miniproject-2/abo-images-smal...,does the item have handle?,yes,yes
46645,41OUfSgfGML,/kaggle/input/vr-miniproject-2/abo-images-smal...,what is this object?,lamp,lamp


In [16]:
from sklearn.metrics import accuracy_score
from bert_score import score as bert_score

# Normalize ground truth and predicted answers
val_df["clean_answer"] = val_df["answer"].astype(str).str.lower().str.strip()
val_df["clean_prediction"] = val_df["predicted_answer"].astype(str).str.lower().str.strip()

# Exact match accuracy
exact_match = accuracy_score(val_df["clean_answer"], val_df["clean_prediction"])
print(f"Exact Match Accuracy: {exact_match:.4f}")

# Filter out empty entries before computing BERTScore
non_empty = val_df[
    (val_df["clean_answer"] != "") & (val_df["clean_prediction"] != "")
]

# Compute BERTScore
P, R, F1 = bert_score(
    cands=non_empty["clean_prediction"].tolist(),
    refs=non_empty["clean_answer"].tolist(),
    lang="en",
    verbose=True
)

print(f"BERTScore — Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

Exact Match Accuracy: 0.2651


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/26 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/125 [00:00<?, ?it/s]

done in 2.91 seconds, 2744.73 sentences/sec
BERTScore — Precision: 0.9659, Recall: 0.9601, F1: 0.9623


In [17]:
import shutil

shutil.make_archive("lora_adapter_blip", 'zip', "lora_adapter_blip")

'/kaggle/working/lora_adapter_blip.zip'